In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *


done1
done2


p(CDR3)=logP(V)+logP(D|J)+logP(J)+logP(delv|v)+logP(delD5|D)+logP(delD3|D)+logP(delJ|J)+logP(ins1)+logP(ins2)

for each sequence:
(1) get identities of each recombination event
(2) calculate the probs for each event, and sum their logs
(3)calculate in how many samples we will expect this sequence to appear 

In [2]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/trainSetSamples','rb') as f1:
    trainSetSamples=pickle.load(f1)
f1.close()

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/TestSetSamples','rb') as f2:
    TestSetSamples=pickle.load(f2)
f2.close()



## at first stage, take only 1 sample from the test set and get the values for relevant parameters:

In [ ]:
#choose the first sample in the test set:
TestSetSamples[0]

In [ ]:
# generate list of all sequences in the sample and the following parameters:
# v,d,j,delv,deld5,deld3,delj,ins1length,ins2length

In [ ]:
#get the relevant parameters for sample HIP13933

from Bio.Seq import Seq

sample_name='HIP13933'
sample_df = pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/%s.tsv" % sample_name) 
sample_df_non_prod = sample_df[sample_df['sequenceStatus'] != 'In']


## v,d,j,delv,deld5,deld3,delj,ins1length,ins2length
RelevantParams_HIP13933=sample_df_non_prod[['nucleotide','vGeneName','dFamilyName','jGeneName','vDeletion','d5Deletion','d3Deletion',
                                             'jDeletion','n1Index','n2Index','dIndex','jIndex']]
df=RelevantParams_HIP13933
for n,i in enumerate(df.index):
    print n
    n1Index=df.loc[i,'n1Index']
    dIndex=df.loc[i,'dIndex']
    n2Index=df.loc[i,'n2Index']
    jIndex=df.loc[i,'jIndex']
    if n1Index == -1 or dIndex == -1:
        df.loc[i,'n1InsSeq']=None
        df.loc[i,'n1InsLen']=None 
    else:
        n1InsSeq = df.loc[i,'nucleotide'][n1Index:dIndex]
        df.loc[i,'n1InsSeq']=n1InsSeq
        df.loc[i,'n1InsLen']=len(n1InsSeq) 
    if n2Index == -1 or jIndex == -1:
        df.loc[i,'n2InsSeq']=None
        df.loc[i,'n2InsLen']=None 
    else:
        n2InsSeqRev = df.loc[i,'nucleotide'][n2Index:jIndex]
        n2InsSeqRev2 = Seq(n2InsSeqRev)
        n2InsSeq = n2InsSeqRev2.reverse_complement()
        n2InsSeq = ''.join(n2InsSeq)
        n2InsLen = len(n2InsSeq)
        df.loc[i,'n2InsSeq']=n2InsSeq
        df.loc[i,'n2InsLen']=n2InsLen


In [ ]:
df.tail()

In [ ]:
#save
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/RelevantParams_HIP13933'
RelevantParams_HIP13933.to_pickle(file1)

In [ ]:
#check duplicates

In [ ]:
#below are checks for duplications. 

#the conclusions are:

#no row is duplicated compleletly. 
#removing the 'nucleotide' parameter, 1402 rows are duplicated.
# Most non-unique combinations occure 2 times, except for 4 combinations that occure 3 times and 1 combination that occurs 4 times. 




In [ ]:
RelevantParams_HIP13933.columns.values

In [ ]:
columns_to_check=['vGeneName', 'dFamilyName', 'jGeneName', 'vDeletion',
       'd5Deletion', 'd3Deletion', 'jDeletion', 'n1InsSeq', 'n2InsSeq']

In [ ]:
trial=RelevantParams_HIP13933[RelevantParams_HIP13933.duplicated(subset=columns_to_check,keep=False)]
trial[(trial['vGeneName']=='TCRBV27-01')&(trial['vDeletion']==11)]
print len(trial)

In [ ]:
#save
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/duplicateRows_RelevantParams_HIP13933.xlsx'
trial.to_excel(file1)

In [ ]:
## see how many replicates for each duplication:
trial.groupby(columns_to_check).count().sort_values(by='nucleotide')

In [ ]:
#generate a similar df for ALL samples in the test set together:

In [ ]:
#define the function for a specific sample:

# this function was copied to eclipse- SuffStat_RunCDR3CalculTestSet.py

def gen_params_summary_df_for_oneSample(sample_name):
    from Bio.Seq import Seq

    print sample_name
    sample_df = pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/%s.tsv" % sample_name) 
    sample_df_non_prod = sample_df[sample_df['sequenceStatus'] != 'In']


    ## v,d,j,delv,deld5,deld3,delj,ins1length,ins2length
    df=sample_df_non_prod[['nucleotide','vGeneName','dFamilyName','jGeneName','vDeletion','d5Deletion','d3Deletion',
                                                 'jDeletion','n1Index','n2Index','dIndex','jIndex']]
    df=df[:20] ## delete when real!
    for n,i in enumerate(df.index):
        if n%5000==0:
            print n
        n1Index=df.loc[i,'n1Index']
        dIndex=df.loc[i,'dIndex']
        n2Index=df.loc[i,'n2Index']
        jIndex=df.loc[i,'jIndex']
        if n1Index == -1 or dIndex == -1:
            df.loc[i,'n1InsSeq']=None
            df.loc[i,'n1InsLen']=None 
        else:
            n1InsSeq = df.loc[i,'nucleotide'][n1Index:dIndex]
            df.loc[i,'n1InsSeq']=n1InsSeq
            df.loc[i,'n1InsLen']=len(n1InsSeq) 
        if n2Index == -1 or jIndex == -1:
            df.loc[i,'n2InsSeq']=None
            df.loc[i,'n2InsLen']=None 
        else:
            n2InsSeqRev = df.loc[i,'nucleotide'][n2Index:jIndex]
            n2InsSeqRev2 = Seq(n2InsSeqRev)
            n2InsSeq = n2InsSeqRev2.reverse_complement()
            n2InsSeq = ''.join(n2InsSeq)
            n2InsLen = len(n2InsSeq)
            df.loc[i,'n2InsSeq']=n2InsSeq
            df.loc[i,'n2InsLen']=n2InsLen
    df['Sample']=sample_name
        
    return df


In [ ]:
#run for 3 samples:

df_list=[]
for n, sample_name in enumerate(TestSetSamples[:3]):
    print n
    df=gen_params_summary_df_for_sampleset(sample_name)
    df_list.append(df)
params_summary_df_for_sampleset=pd.concat(df_list)
params_summary_df_for_sampleset = params_summary_df_for_sampleset.set_index('Sample')
params_summary_df_for_sampleset.index = params_summary_df_for_sampleset.index.str.replace("HIP", "")
    

calculate for each sequence:
1. predicted frequency of value combinations by train set
2. predicted frequency of value combination by test set
2. observed frequency of value combination in test set
4. observed frequency of value combination in train set

In [ ]:
valueCombinations_HIP13933=RelevantParams_HIP13933.groupby(columns_to_check).count()

In [ ]:
valueCombinations_HIP13933.rename(columns={'nucleotide':'count'},inplace=True)

In [ ]:
valueCombinations_HIP13933.head()

In [ ]:
valueCombinations_HIP13933['count'].sum()

In [ ]:
valueCombinations_HIP13933['count'].count()

In [ ]:
valueCombinations_HIP13933['frequency']=valueCombinations_HIP13933['count']/valueCombinations_HIP13933['count'].sum()

In [ ]:
valueCombinations_HIP13933.index[0]

In [ ]:
list(valueCombinations_HIP13933.index.names)

In [ ]:
firstValueCombination=valueCombinations_HIP13933.index[0]

## defining functions needed to calculate the log prob of a specific parameters combination:

In [ ]:
#calculate the sum of logs of conditional probs for a specific combination:

def calc_cond_prob(indParVal, depParVal,condProbDF):
    
    CondProb=condProbDF.loc[indParVal,depParVal]
    logCondProb=np.log10(CondProb)
    return logCondProb

def calc_all_condProbs(seqParamsValDict,setType):
    indPar_list=['jGeneName','vGeneName','dFamilyName','dFamilyName','jGeneName']
    depPar_list=['dFamilyName','vDeletion','d5Deletion','d3Deletion','jDeletion']
    condProb_Train_df_list=[condProb_jGeneName_dFamilyName_Train_293,condProb_vGeneName_vDeletion_Train_293,
                           condProb_dFamilyName_d5Deletion_Train_293,condProb_dFamilyName_d3Deletion_Train_293,
                           condProb_jGeneName_jDeletion_Train_293]
    condProb_Test_df_list=[condProb_jGeneName_dFamilyName_Test_294,condProb_vGeneName_vDeletion_Test_294,
                           condProb_dFamilyName_d5Deletion_Test_294,condProb_dFamilyName_d3Deletion_Test_294,
                           condProb_jGeneName_jDeletion_Test_294]

    totalLogCondPro=0   
    
    for n in range(len(indPar_list)):
        indPar=indPar_list[n]
        depPar=depPar_list[n]
        #print indPar,depPar
        indParVal=seqParamsValDict[indPar]
        depParVal=seqParamsValDict[depPar]
        if setType=='Train':
            condProbDF=condProb_Train_df_list[n]
        else:
            condProbDF=condProb_Test_df_list[n]
        logCondPro=calc_cond_prob(indParVal, depParVal,condProbDF)
        totalLogCondPro=totalLogCondPro+logCondPro


    return totalLogCondPro
    


In [ ]:
def load_all_train_probs_files():
    
    
    global vGeneTrainProbs_294,jGeneTrainProbs_294,condProb_jGeneName_dFamilyName_Train_293, \
            condProb_vGeneName_vDeletion_Train_293,condProb_jGeneName_jDeletion_Train_293, \
            condProb_dFamilyName_d5Deletion_Train_293,condProb_dFamilyName_d3Deletion_Train_293, \
            nt1FreqsDictDict_Train_ins1, lengthCount_Train_ins1, dinucNormDFDict_Train_ins1,nt1FreqsDictDict_Train_ins2,  \
            lengthCount_Train_ins2, dinucNormDFDict_Train_ins2
    
    
    #V:
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/vGeneTrainProbs_294'
    vGeneTrainProbs_294=pd.read_pickle(file1)

    #J:
    file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/jGeneTrainProbs_294'
    jGeneTrainProbs_294=pd.read_pickle(file2)

    #D|J:
    file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/condProb_jGeneName_dFamilyName_Train_293'
    condProb_jGeneName_dFamilyName_Train_293=pd.read_pickle(file3)

    #delV|V:
    file4='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/condProb_vGeneName_vDeletion_Train_293'
    condProb_vGeneName_vDeletion_Train_293=pd.read_pickle(file4)

    #delD5|D:
    file5='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/condProb_dFamilyName_d5Deletion_Train_293'
    condProb_dFamilyName_d5Deletion_Train_293=pd.read_pickle(file5)

    #delD3|D:
    file6='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/condProb_dFamilyName_d3Deletion_Train_293'
    condProb_dFamilyName_d3Deletion_Train_293=pd.read_pickle(file6)

    #delJ|J:
    file7='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/condProb_jGeneName_jDeletion_Train_293'
    condProb_jGeneName_jDeletion_Train_293=pd.read_pickle(file7)
    
    #ins1, ins2 parameters:

    file11='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/nt1FreqsDictDict_train_ins1'
    nt1FreqsDictDict_Train_ins1=pd.read_pickle(file11)

    file12='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/nt1FreqsDictDict_train_ins2'
    nt1FreqsDictDict_Train_ins2=pd.read_pickle(file12)

    file13='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/lengthCount_train_ins1'
    lengthCount_Train_ins1=pd.read_pickle(file13)

    file14='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/lengthCount_train_ins2'
    lengthCount_Train_ins2=pd.read_pickle(file14)

    file15='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/dinucNormDFDict_train_ins1'
    dinucNormDFDict_Train_ins1=pd.read_pickle(file15)

    file16='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/dinucNormDFDict_train_ins2'
    dinucNormDFDict_Train_ins2=pd.read_pickle(file16)
    





In [ ]:
def load_all_test_probs_files():
    
    global vGeneTestProbs_293,jGeneTestProbs_293,condProb_jGeneName_dFamilyName_Test_294, \
    condProb_vGeneName_vDeletion_Test_294,     condProb_jGeneName_jDeletion_Test_294, \
    condProb_dFamilyName_d5Deletion_Test_294,condProb_dFamilyName_d3Deletion_Test_294, \
    nt1FreqsDictDict_Test_ins1, lengthCount_Test_ins1, dinucNormDFDict_Test_ins1,nt1FreqsDictDict_Test_ins2, \
    lengthCount_Test_ins2,     dinucNormDFDict_Test_ins2
    
    #V:
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/vGeneTestProbs_293'
    vGeneTestProbs_293=pd.read_pickle(file1)

    #J:
    file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/jGeneTestProbs_293'
    jGeneTestProbs_293=pd.read_pickle(file2)

    #D|J:
    file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/condProb_jGeneName_dFamilyName_Test_294'
    condProb_jGeneName_dFamilyName_Test_294=pd.read_pickle(file3)

    #delV|V:
    file4='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/condProb_vGeneName_vDeletion_Test_294'
    condProb_vGeneName_vDeletion_Test_294=pd.read_pickle(file4)

    #delD5|D:
    file5='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/condProb_dFamilyName_d5Deletion_Test_294'
    condProb_dFamilyName_d5Deletion_Test_294=pd.read_pickle(file5)

    #delD3|D:
    file6='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/condProb_dFamilyName_d3Deletion_Test_294'
    condProb_dFamilyName_d3Deletion_Test_294=pd.read_pickle(file6)

    #delJ|J:
    file7='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/condProb_jGeneName_jDeletion_Test_294'
    condProb_jGeneName_jDeletion_Test_294=pd.read_pickle(file7)
    
    #ins1, ins2 parameters:

    file11='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/nt1FreqsDictDict_test_ins1'
    nt1FreqsDictDict_Test_ins1=pd.read_pickle(file11)

    file12='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/nt1FreqsDictDict_test_ins2'
    nt1FreqsDictDict_Test_ins2=pd.read_pickle(file12)

    file13='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/lengthCount_test_ins1'
    lengthCount_Test_ins1=pd.read_pickle(file13)

    file14='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/lengthCount_test_ins2'
    lengthCount_Test_ins2=pd.read_pickle(file14)

    file15='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/dinucNormDFDict_test_ins1'
    dinucNormDFDict_Test_ins1=pd.read_pickle(file15)

    file16='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/dinucNormDFDict_test_ins2'
    dinucNormDFDict_Test_ins2=pd.read_pickle(file16)


In [ ]:
# a function calculating the probability of a specific combination based on either the train or test data set:
#when generalizing this function. need to take the read_pickles out

def calc_logProb_per_seq(seqParamsValDict,setType):
    totalLogProb=0
       
    load_all_train_probs_files()
    load_all_test_probs_files()
       
    
    #calculate sum of logs of conditional probs and add to the total log prob:
    totalLogCondPro=calc_all_condProbs(seqParamsValDict,setType)
    totalLogProb=+totalLogCondPro
    
    #calc log probs for v and j and add to the total log prob:
    if setType=='Train':
        vGeneProbsDF=vGeneTrainProbs_294
        jGeneProbsDF=jGeneTrainProbs_294
    else: 
        vGeneProbsDF=vGeneTestProbs_293
        jGeneProbsDF=jGeneTestProbs_293          
       
    vGeneName=seqParamsValDict['vGeneName']
    vProb=vGeneProbsDF[vGeneName]
    #print 'vprob=%s' %vProb
    logVProb=np.log10(vProb)
    
    jGeneName=seqParamsValDict['jGeneName']
    jProb=jGeneProbsDF[jGeneName]
    #print 'jprob=%s' %jProb
    logJProb=np.log10(jProb)
   
    totalLogProb=totalLogProb+logVProb+logJProb
    
    #calc log probs for ins1 and ins2 and add to the total log prob:
    if setType=='Train':
        n1InsSeq=seqParamsValDict['n1InsSeq']
        logProbins1Seq=calc_insSeq_prob(n1InsSeq, 20, nt1FreqsDictDict_Train_ins1, dinucNormDFDict_Train_ins1, lengthCount_Train_ins1)
        n2InsSeq=seqParamsValDict['n2InsSeq']
        logProbins2Seq=calc_insSeq_prob(n2InsSeq, 20, nt1FreqsDictDict_Train_ins2, dinucNormDFDict_Train_ins2, lengthCount_Train_ins2)
    else:
        n1InsSeq=seqParamsValDict['n1InsSeq']
        logProbins1Seq=calc_insSeq_prob(n1InsSeq, 20, nt1FreqsDictDict_Test_ins1, dinucNormDFDict_Test_ins1, lengthCount_Test_ins1)
        n2InsSeq=seqParamsValDict['n2InsSeq']
        logProbins2Seq=calc_insSeq_prob(n2InsSeq, 20, nt1FreqsDictDict_Test_ins2, dinucNormDFDict_Test_ins2, lengthCount_Test_ins2)
        
        
        
    totalLogProb=totalLogProb+logProbins1Seq+logProbins2Seq
    
    #print 'ins1 prob=%s' %10**logProbins1Seq
    #print 'ins2 prob=%s' %10**logProbins2Seq
    
    return totalLogProb
    
    

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/params_summary_tables/params_summary_TestSet_grouped_replicates'
params_summary_TestSet_grouped_replicates=pd.read_pickle(file1)

In [ ]:
n=0
seqParamsVals=params_summary_TestSet_grouped_replicates.index[n]
paramsNames=params_summary_TestSet_grouped_replicates.index.names

seqParamsValDict=dict(zip(paramsNames, seqParamsVals))

In [ ]:
totalLogProb=calc_logProb_per_seq(seqParamsValDict,'Test')
totalLogProb

In [ ]:
totalLogProb=calc_logProb_per_seq(seqParamsValDict,'Train')
totalLogProb

# generating a df with all combinations in the test set
# generating a df with all combinations in the test set *that repeat more than once*:

In [ ]:
#the function gen_params_summary_df_for_oneSample(sample_name) was run for all samples using the eclipse (parallel runs)
# the resulting dfs needs to be concatenate to one:

dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/params_summary_tables/gen_params_summary_df_for_sampleset'
params_summary_TestSet=concat_summarizing_dfs(dfs_folder)

In [ ]:
params_summary_TestSet.head()

In [ ]:
params_summary_TestSet.info()

In [ ]:
#save the complete df
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/params_summary_tables/params_summary_TestSet'
params_summary_TestSet.to_pickle(file1)

In [ ]:
#save the complete df
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/params_summary_tables/params_summary_TestSet'
params_summary_TestSet=pd.read_pickle(file1)

In [ ]:
# generate df of all possible parameters combinations and their count:
params_summary_TestSet_grouped=params_summary_TestSet.groupby(columns_to_check).count()

In [ ]:
params_summary_TestSet_grouped.rename(columns={'nucleotide':'count'},inplace=True)

In [ ]:
params_summary_TestSet_grouped.drop(['n1Index','n2Index','dIndex','jIndex','n1InsLen','n2InsLen'],axis=1,inplace=True)

In [ ]:
params_summary_TestSet_grouped.head()

In [ ]:
params_summary_TestSet_grouped.sort_values(by=['count'],ascending=False,inplace=True)

In [ ]:
params_summary_TestSet_grouped.info()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/params_summary_tables/params_summary_TestSet_grouped'
params_summary_TestSet_grouped.to_pickle(file1)

In [ ]:
print len(params_summary_TestSet_grouped.index)

In [ ]:
# generate a sub-df which contains only combinations that repeat more than once in the test set (~2% of the combinations)

params_summary_TestSet_grouped_replicates=params_summary_TestSet_grouped[params_summary_TestSet_grouped['count']>1]
print len(params_summary_TestSet_grouped_replicates)

In [ ]:
171100./8352567

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/params_summary_tables/params_summary_TestSet_grouped_replicates'
params_summary_TestSet_grouped_replicates.to_pickle(file1)

# run on each repeated combination and check its predicted and observed freqs:

a function to run on each combination in params_summary_TestSet_grouped_replicates
calculates:
1. predicted freq by the train set
2. predicted freq by the test set
3. observed freq in the test set

In [ ]:
def compare_cdr3_prediction(min_comb,max_comb,df):

    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/params_summary_tables/params_summary_TestSet_grouped_replicates'
    params_summary_TestSet_grouped_replicates=pd.read_pickle(file1)
        
    n_combinations_testSet=8352567
    
    
    n_list=[]
    totalLogProb_Train_list=[]
    totalLogProb_Test_list=[]
    logObsFreqInTestSet_list=[]
   
    for n in range(min_comb,max_comb):
        print n
        seqParamsVals=df.index[n]
        paramsNames=df.index.names
        seqParamsValDict=dict(zip(paramsNames, seqParamsVals))
        
        totalLogProb_Test=calc_logProb_per_seq(seqParamsValDict,'Test')
        totalLogProb_Train=calc_logProb_per_seq(seqParamsValDict,'Train')
        obsfreq=float(params_summary_TestSet_grouped_replicates['count'][n])/n_combinations_testSet
        logObsFreqInTestSet=np.log10(obsfreq)
        
        
        #exp_n_comb_appears_in_sample=n_combinations_sample*10**totalLogProb
        exp_n_comb_appears_in_testSet=n_combinations_testSet*10**totalLogProb
        real_n_in_testSet=params_summary_TestSet_grouped_replicates['count'][n]
        
        
        n_list.append(n)
        totalLogProb_Train_list.append(totalLogProb_Train)
        totalLogProb_Test_list.append(totalLogProb_Test)
        logObsFreqInTestSet_list.append(logObsFreqInTestSet)

    cdr3_prediction_compare_df=pd.DataFrame({'n':n_list, 'cdr3LogProb_byTrainSet':totalLogProb_Train_list,
                                        'cdr3LogProb_byTestSet':totalLogProb_Test_list,
                                        'cdr3LogObsFreq': logObsFreqInTestSet_list})
        
        
        
    return cdr3_prediction_compare_df


In [ ]:
7.0/50

In [ ]:
0.14*170000/3600

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/params_summary_tables/params_summary_TestSet_grouped'
params_summary_TestSet_grouped=pd.read_pickle(file1)

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/params_summary_tables/params_summary_TestSet_grouped_replicates'
params_summary_TestSet_grouped_replicates=pd.read_pickle(file1)

In [ ]:
print len(params_summary_TestSet_grouped_replicates.index)

In [ ]:
%%time
df=params_summary_TestSet_grouped_replicates
df2=params_summary_TestSet_grouped
cdr3_prediction_compare_df=compare_cdr3_prediction(0,20,df)

In [ ]:
170000/2000

In [ ]:
cdr3_prediction_compare_df

In [ ]:
x=cdr3_prediction_compare_df['cdr3LogProb_byTrainSet']
y=cdr3_prediction_compare_df['cdr3LogProb_byTestSet']
z=cdr3_prediction_compare_df['cdr3LogObsFreq']

In [ ]:
#correl between test and train set predictions:
r,p=MyPearsonr(x,y)
print r,p

In [ ]:
#correl between train prediction and observed :
r,p=MyPearsonr(x,z)
print r,p

In [ ]:
#correl between test prediction and observed :
r,p=MyPearsonr(y,z)
print r,p

In [ ]:
print '%.5f' %x

## combine the results for all combinations in the test set that appears more than once:

the function compare_cdr3_prediction was run on all repeating combinations (171100 combinations summarized in 'params_summary_TestSet_grouped_replicates')

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/compare_cdr3_prediction'
compare_cdr3_prediction_allRepeatingSeqs=concat_summarizing_dfs(dfs_folder)

In [ ]:
compare_cdr3_prediction_allRepeatingSeqs['logFoldDiffObsToPredict']=compare_cdr3_prediction_allRepeatingSeqs['cdr3LogObsFreq']-compare_cdr3_prediction_allRepeatingSeqs['cdr3LogProb_byTrainSet']

In [ ]:
compare_cdr3_prediction_allRepeatingSeqs.head()

In [ ]:
compare_cdr3_prediction_allRepeatingSeqs.describe()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/params_summary_tables/compare_cdr3_prediction_allRepeatingSeqs'
compare_cdr3_prediction_allRepeatingSeqs.to_pickle(file1)

In [3]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/params_summary_tables/compare_cdr3_prediction_allRepeatingSeqs'
compare_cdr3_prediction_allRepeatingSeqs=pd.read_pickle(file1)

In [ ]:
#look for the sequences with the higher predicted frequency:
highestPredictionsDF=compare_cdr3_prediction_allRepeatingSeqs[compare_cdr3_prediction_allRepeatingSeqs['cdr3LogProb_byTrainSet']>-8]

In [ ]:
params_summary_TestSet_grouped_replicates.index[159024]

In [ ]:
highestPredictionsDF.head()

In [ ]:
for n in highestPredictionsDF.index:
    print params_summary_TestSet_grouped_replicates.index[n]


In [ ]:
#look for the sequences with the lowest predicted frequency:
LowestPredictionsDF=compare_cdr3_prediction_allRepeatingSeqs[compare_cdr3_prediction_allRepeatingSeqs['cdr3LogProb_byTrainSet']<-42]

In [ ]:
LowestPredictionsDF

In [ ]:
LowestPredictionsDF=LowestPredictionsDF.set_index('n')
for n in LowestPredictionsDF.index:
    print params_summary_TestSet_grouped_replicates.index[n]

In [ ]:
#plot correlation between predictions by train and set session-only high!

df=highestPredictionsDF
x_var = 'cdr3LogProb_byTrainSet'
y_var = 'cdr3LogProb_byTestSet'

x=df[x_var]
y=df[y_var]
ymean=df[y_var].mean()
filename = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/SuffStat Images/CDR3_freqs_predictions_correl-highestPredictionsDF'



draw_correlation_scatter_onePlotperFigure(x, y, figsize = (3, 3), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = 'pearson', ms=4, logd = False,\
                             xlab = x_var, ylab = y_var, filename = filename, title = None,
                             color = "#a0a0a0", grid = True, dpi = 200, xticklabels = None, 
                             xticklabelsSize=7, yticklabelsSize=7, xticksAlign=None, contour = False)
# fit with np.polyfit
nx=np.isnan(x)
ny=np.isnan(y)
n=nx+ny
newx=list(x[~n])
newy=list(y[~n])
plt.plot(np.unique(newx), np.poly1d(np.polyfit(newx, newy, 1))(np.unique(newx)))
plt.plot(x=x,c='black', linewidth=4)


plt.show()

In [ ]:
#plot correlation between predictions by train and set session-first 5000 combinations:

df=compare_cdr3_prediction_allRepeatingSeqs[:5000]
x_var = 'cdr3LogProb_byTrainSet'
y_var = 'cdr3LogProb_byTestSet'

x=df[x_var]
y=df[y_var]
ymean=df[y_var].mean()
filename = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/SuffStat Images/CDR3_freqs_predictions_correl-first5000combinations'



draw_correlation_scatter_onePlotperFigure(x, y, figsize = (3, 3), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = 'pearson', ms=4, logd = False,\
                             xlab = x_var, ylab = y_var, filename = filename, title = None,
                             color = "#a0a0a0", grid = True, dpi = 200, xticklabels = None, 
                             xticklabelsSize=7, yticklabelsSize=7, xticksAlign=None, contour = False)
# fit with np.polyfit
nx=np.isnan(x)
ny=np.isnan(y)
n=nx+ny
newx=list(x[~n])
newy=list(y[~n])
plt.plot(np.unique(newx), np.poly1d(np.polyfit(newx, newy, 1))(np.unique(newx)))
plt.plot(x=x,c='black', linewidth=4)


plt.show()

In [ ]:
#plot correlation between predictions by train and set session

df=highestPredictionsDF
x_var = 'cdr3LogProb_byTrainSet'
y_var = 'cdr3LogObsFreq'

x=df[x_var]
y=df[y_var]
ymean=df[y_var].mean()
filename = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/SuffStat Images/CDR3_freqs_obs_to_predict_correl-highestPredictionsDF'




draw_correlation_scatter_onePlotperFigure(x, y, figsize = (3, 3), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = 'pearson', ms=4, logd = False,\
                             xlab = x_var, ylab = y_var, filename = filename, title = None,
                             color = "#a0a0a0", grid = True, dpi = 200, xticklabels = None, 
                             xticklabelsSize=7, yticklabelsSize=7, xticksAlign=None, contour = False)
# fit with np.polyfit
nx=np.isnan(x)
ny=np.isnan(y)
n=nx+ny
newx=list(x[~n])
newy=list(y[~n])
plt.plot(np.unique(newx), np.poly1d(np.polyfit(newx, newy, 1))(np.unique(newx)))
plt.plot(x=x,c='black', linewidth=4)


plt.show()

In [ ]:
#plot correlation between predictions by train and set session, first 5000 combinations:

df=compare_cdr3_prediction_allRepeatingSeqs[-5000:]
x_var = 'cdr3LogProb_byTrainSet'
y_var = 'cdr3LogObsFreq'

x=df[x_var]
y=df[y_var]
ymean=df[y_var].mean()
filename = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/SuffStat Images/CDR3_freqs_obs_to_predict_correl-first5000combinations'



draw_correlation_scatter_onePlotperFigure(x, y, figsize = (3, 3), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = 'pearson', ms=4, logd = False,\
                             xlab = x_var, ylab = y_var, filename = filename, title = None,
                             color = "#a0a0a0", grid = True, dpi = 200, xticklabels = None, 
                             xticklabelsSize=7, yticklabelsSize=7, xticksAlign=None, contour = False)
# fit with np.polyfit
nx=np.isnan(x)
ny=np.isnan(y)
n=nx+ny
newx=list(x[~n])
newy=list(y[~n])
plt.plot(np.unique(newx), np.poly1d(np.polyfit(newx, newy, 1))(np.unique(newx)))
plt.plot(x=x,c='black', linewidth=4)


plt.show()

In [ ]:
compare_cdr3_prediction_allRepeatingSeqs.head()

In [4]:
df=compare_cdr3_prediction_allRepeatingSeqs.sort_values(by='cdr3LogObsFreq')[-5000:]

In [5]:
df

,cdr3LogObsFreq,cdr3LogProb_byTestSet,cdr3LogProb_byTrainSet,n,logFoldDiffObsToPredict
1970,-6.620790,-13.671936,-13.671185,105970,7.050395
1968,-6.620790,-17.000765,-17.006995,105968,10.386205
1954,-6.620790,-15.861177,-15.846868,105954,9.226078
1955,-6.620790,-13.604174,-13.587427,105955,6.966637
1956,-6.620790,-11.569301,-11.588902,105956,4.968112
1957,-6.620790,-14.527693,-14.515023,105957,7.894233
1958,-6.620790,-15.357657,-15.362452,105958,8.741662
1959,-6.620790,-29.904584,-29.858379,105959,23.237589
1969,-6.620790,-13.081124,-13.116516,105969,6.495726
1960,-6.620790,-12.853359,-12.890930,105960,6.270140


In [6]:
ax=None

df=df

In [7]:
x_var = 'cdr3LogProb_byTrainSet'
y_var = 'cdr3LogObsFreq'

In [8]:
x=df[x_var]
y=df[y_var]

In [9]:
ymean=df[y_var].mean()
filename = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/SuffStat Images/CDR3_freqs_obs_to_predict_correl-first5000combinations'
print 'defined variables...'

defined variables...


In [10]:
fig = plt.figure()
if ax is None:
    ax = fig.add_subplot(111)

ax.plot(x,y, marker='o', linestyle='', ms=6) 
#label=name,c=color_list[count])

print 'plotted graph...'


plotted graph...


In [11]:
ax.set_xlabel(x_var)
ax.set_ylabel(y_var)
plt.suptitle('CDR3_freqs_obs_to_predict_correl\n First5000combinations',fontsize=14)

#ax.legend(loc=1)
print 'added labels...'

added labels...


In [ ]:
# fit with np.polyfit
nx=np.isnan(x)
ny=np.isnan(y)
n=nx+ny
newx=list(x[~n])
newy=list(y[~n])
ax.plot(np.unique(newx), np.poly1d(np.polyfit(newx, newy, 1))(np.unique(newx)),c='black')
ax.plot(x=x,c='black', linewidth=4)

print 'fitted regression line...'

In [ ]:
#get positions for text:

ylim=ax.get_ylim()
ypos=ylim[1]
xlim=ax.get_xlim()
xpos=xlim[0]
        
#calculate pearson:
r,p=MyPearsonr(x,y)

print 'calculated pearson...'

In [ ]:
ax.text(xpos,ypos,"r=%.4f p=%.6f" %(r,p),  verticalalignment = 'top', ha = 'left',fontsize=14,color='red')
fig.subplots_adjust(top=0.8)
plt.show()

In [ ]:
ax=None

df=df
x_var = 'cdr3LogProb_byTrainSet'
y_var = 'cdr3LogObsFreq'

x=df[x_var]
y=df[y_var]
ymean=df[y_var].mean()
filename = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/SuffStat Images/CDR3_freqs_obs_to_predict_correl-first5000combinations'
print 'defined variables...'

fig = plt.figure()
if ax is None:
    ax = fig.add_subplot(111)

ax.plot(x,y, marker='o', linestyle='', ms=6) 
#label=name,c=color_list[count])

print 'plotted graph...'

ax.set_xlabel(x_var)
ax.set_ylabel(y_var)
plt.suptitle('CDR3_freqs_obs_to_predict_correl\n First5000combinations',fontsize=14)

#ax.legend(loc=1)
print 'added labels...'

# fit with np.polyfit
nx=np.isnan(x)
ny=np.isnan(y)
n=nx+ny
newx=list(x[~n])
newy=list(y[~n])
ax.plot(np.unique(newx), np.poly1d(np.polyfit(newx, newy, 1))(np.unique(newx)),c='black')
ax.plot(x=x,c='black', linewidth=4)

print 'fitted regression line...'
        
#get positions for text:

ylim=ax.get_ylim()
ypos=ylim[1]
xlim=ax.get_xlim()
xpos=xlim[0]
        
#calculate pearson:
r,p=MyPearsonr(x,y)

print 'calculated pearson...'
    

ax.text(xpos,ypos,"r=%.4f p=%.6f" %(r,p),  verticalalignment = 'top', ha = 'left',fontsize=14,color='red')
fig.subplots_adjust(top=0.8)
plt.show()

show correlation plots in the ppt and summarize
go to igur:
predict events probabilities for all sequences in a specific sample and then use this sample for 'within samples' analysis.
if plausible, predict events probabilities for all samples, and use 'between samples 0.5:0.5' method to repeat the analysis.
try to predict productive sequences by the non-productive model (start with one sample!)

go to the 'non-productive' clonality issues...
predicting ethnicity or age by the repertoire?